In [1]:
import numpy as np

In [2]:
test0 = ['x00: 1',
         'x01: 1',
         'x02: 1',
         'y00: 0',
         'y01: 1',
         'y02: 0',
         '',
         'x00 AND y00 -> z00',
         'x01 XOR y01 -> z01',
         'x02 OR y02 -> z02']

test1 = ['x00: 1',
         'x01: 0',
         'x02: 1',
         'x03: 1',
         'x04: 0',
         'y00: 1',
         'y01: 1',
         'y02: 1',
         'y03: 1',
         'y04: 1',
         '',
         'ntg XOR fgs -> mjb',
         'y02 OR x01 -> tnw',
         'kwq OR kpj -> z05',
         'x00 OR x03 -> fst',
         'tgd XOR rvg -> z01',
         'vdt OR tnw -> bfw',
         'bfw AND frj -> z10',
         'ffh OR nrd -> bqk',
         'y00 AND y03 -> djm',
         'y03 OR y00 -> psh',
         'bqk OR frj -> z08',
         'tnw OR fst -> frj',
         'gnj AND tgd -> z11',
         'bfw XOR mjb -> z00',
         'x03 OR x00 -> vdt',
         'gnj AND wpb -> z02',
         'x04 AND y00 -> kjc',
         'djm OR pbm -> qhw',
         'nrd AND vdt -> hwm',
         'kjc AND fst -> rvg',
         'y04 OR y02 -> fgs',
         'y01 AND x02 -> pbm',
         'ntg OR kjc -> kwq',
         'psh XOR fgs -> tgd',
         'qhw XOR tgd -> z09',
         'pbm OR djm -> kpj',
         'x03 XOR y03 -> ffh',
         'x00 XOR y04 -> ntg',
         'bfw OR bqk -> z06',
         'nrd XOR fgs -> wpb',
         'frj XOR qhw -> z04',
         'bqk OR frj -> z07',
         'y03 OR x01 -> nrd',
         'hwm AND bqk -> z03',
         'tgd XOR rvg -> z12',
         'tnw OR pbm -> gnj']

In [3]:
def get_wires(data):
    wires = {}
    device = []
    output = []
    
    is_wire = True
    for line in data:
        line = line.strip()
        if len(line) == 0:
            is_wire = False
            
        elif is_wire:
            wire, value = line.split(': ')
            wires[wire] = bool(int(value))
            
        else:
            parts = line.split(' ')
            device.append(parts)
            if parts[-1][0] == 'z':
                output.append(parts[-1])
            
    output.sort(reverse=True)
    return wires, device, output

def run_device(wires, device):
    
    while len(device):
    
        for i in range(len(device)-1, -1, -1):
            gate = device[i]
            if gate[0] in wires and gate[2] in wires and gate[4] not in wires:
                if gate[1] == 'AND':
                    wires[gate[4]] = wires[gate[0]] and wires[gate[2]]
                elif gate[1] == 'OR':
                    wires[gate[4]] = wires[gate[0]] or wires[gate[2]]
                elif gate[1] == 'XOR':
                    wires[gate[4]] = wires[gate[0]] != wires[gate[2]]

                del device[i]
                
    return wires

def device_output(wires, output):
    string = ''
    for op in output:
        string += str(int(wires[op]))
    return string

def part1(data):
    wires, device, output = get_wires(data)
    wires = run_device(wires, device)
    output = device_output(wires, output)
    
    print('Part 1 result:', int(output, 2))
    
part1(test0)
part1(test1)

Part 1 result: 4
Part 1 result: 2024


In [4]:
with open('input_day24.txt', 'r') as f:
    data = f.readlines()
    f.close()
    
part1(data)

Part 1 result: 58740594706150


In [5]:
#00 is a half adder, everything else is full adder

def half_adder(device, x, y, bad_wires=[], num='00'):
    
    XOR = None
    AND = None
    
    if x is None:
        x = y
        y = None
    
    idx = np.where(np.logical_or(device[:,0]==x, device[:,2]==x))[0]
    for i in idx:
        if y is not None and y not in device[i]:
            if y not in bad_wires:
                bad_wires.append(y)
            if device[i][0] == x and device[i][2] not in bad_wires:
                bad_wires.append(device[i][2])
            elif device[i][2] == x and device[i][0] not in bad_wires:
                bad_wires.append(device[i][0])
            
        if device[i][1] == 'XOR':
            XOR = device[i][-1]
        elif device[i][1] == 'AND':
            AND = device[i][-1]
            
    return XOR, AND, bad_wires

def OR_logic(device, x, y, bad_wires):
    OR = None
    
    if x is None:
        x = y
        y = None
        
    idx = np.where(np.logical_or(device[:,0]==x, device[:,2]==x))[0]
    for i in idx:
        if y is not None and y not in device[i]:
            if y not in bad_wires:
                bad_wires.append(y)
            if device[i][0] == x and device[i][2] not in bad_wires:
                bad_wires.append(device[i][2])
            elif device[i][2] == x and device[i][0] not in bad_wires:
                bad_wires.append(device[i][0])
            
        if device[i][1] == 'OR':
            OR = device[i][-1]
            
    return OR, bad_wires

def count_next(device, XOR1, AND1, bad_wires, num):
    if XOR1 is None or AND1 is None:
        return XOR1, AND1, bad_wires
    
    idx_XOR1 = np.where(device==XOR1)
    idx_AND1 = np.where(device==AND1)
    
    if len(idx_XOR1[0]) == 2 and len(idx_AND1[0]) == 3:
        if XOR1 not in bad_wires:
            bad_wires.append(XOR1)
        if AND1 not in bad_wires:
            bad_wires.append(AND1)
        return AND1, XOR1, bad_wires
    
    return XOR1, AND1, bad_wires
    

def full_adder(num, device, carry, bad_wires):
    last_num = str(int(num)-1).zfill(2)
    
    x = 'x'+num
    y = 'y'+num
    XOR1, AND1, bad_wires = half_adder(device, x, y, bad_wires, num)
    
    if XOR1 is not None and 'z' in XOR1 and XOR1 not in bad_wires:
        bad_wires.append(XOR1)
    if AND1 is not None and 'z' in AND1 and AND1 not in bad_wires:
        bad_wires.append(AND1)
        
    XOR1, AND1, bad_wires = count_next(device, XOR1, AND1, bad_wires, num)
    
    x = XOR1
    y = carry[last_num]
        
    XOR2, AND2, bad_wires = half_adder(device, x, y, bad_wires, num)
    
    if XOR2 is not None and 'z' not in XOR2 and XOR2 not in bad_wires: #XOR2 is the output
        bad_wires.append(XOR2)
    if AND2 is not None and 'z' in AND2 and AND2 not in bad_wires:
        bad_wires.append(AND2)
    
    x = AND1
    y = AND2
    OR, bad_wires = OR_logic(device, x, y, bad_wires)
    
    if OR is not None and 'z' in OR and OR not in bad_wires:
        bad_wires.append(OR)
    carry[num] = OR
    
    return carry, bad_wires
                

def part2(data):
    wires, device, output = get_wires(data)
    output.sort()
    device = np.array(device)
    
    #00 is a half adder
    XOR, AND, bad_wires = half_adder(device, 'x00', 'y00')
    carry = {'00':AND}
    
    for op in output[1:]:
        carry, bad_wires = full_adder(op[1:], device, carry, bad_wires)

    bad_wires = bad_wires[:-1]
    bad_wires.sort()
    print('Part 2 result:', ','.join(bad_wires))
    
with open('input_day24.txt', 'r') as f:
    data = f.readlines()
    f.close()
    
part2(data)

Part 2 result: cvh,dbb,hbk,kvn,tfn,z14,z18,z23
